# Persisting DataFrame Contents  to Sqlite in Pandas

Pandas is an amazing tool for data analysis.  SQLite is arguably the most commonly deployed SQL database (running at the core of Android Apps).  SQLite does not require a server, because the database is entirely file based - perfect for small databases.  It would be nice to have a simple one-liner interface to persist Pandas DataFrames to and from SQLite databases.  SQLite databases can be read in most languages on virtually all computing platforms, so it is truly a universal data exchange format.  

This notebook provides a brief introduction to writing to, and reading from, SQLite databases from within Pandas.  It does not attempt to provide a detailed coverage of the topic nor does it cover the full spectrum of Pandas I/O options.  A class is described here that provides simple persistence of Pandas DataFrames to a SQLite database.  This class is only a very thin layer around the native Python code.

The Pandas dataframe column names must be the same as the SQLite table column names for this class to work. Ideally the table names and dataframe names should also be the same, but this is not a key requirement.  It stands to reason that the SQLite table definitiom must match the data types in the dataframe, i.e., an alpha string cannot be written to an integer column.

Thanks to @ToobLippe for pointing out that Pandas can write to existing tables as well as create new tables when saving.

In [1]:
from __future__ import print_function

import sqlite3
import pandas as pd

The dBase class provides basic access to open a database and run queries on the database.

In [2]:
class dBase:
    """SQLite database class with minimal operations.
    """
    
    def __init__(self, filename):
        """Define the database to be used and establish a connection to the database.

        Args:
            | filename (string): path/filename to the database.

        Returns:
            | Nothing.

        Raises:
            | None.
        """
        #
        self.db = sqlite3.connect(filename)

    ###########################
    def execSQLCur(self, c,strSQL):
        """Execute a SQL string using the cursor passed as function parameter.

        Args:
            | c (SQLite cursor): the cursor to be used in the operation.
            | strSQL (string): SQL command to tbe executed.

        Returns:
            | Nothing.

        Raises:
            | None.
        """
        strSQL=strSQL.rstrip('\n')
        c.execute(strSQL)

    ###########################
    def execSQL(self, strSQL):
        """execute a SQL string using a local/hidden cursor.

        Args:
            | strSQL (string): SQL command to tbe executed.

        Returns:
            | Nothing.

        Raises:
            | None.
        """
        strSQL=strSQL.rstrip('\n')
        self.db.cursor().execute(strSQL)

    ###########################
    def queryToString(self, strSQL):
        """Return a query result as a string.

        Args:
            | strSQL (string): SQL command to tbe executed.

        Returns:
            | (string): results from the query as a string.

        Raises:
            | None.
        """
        strSQL=strSQL.rstrip('\n')
        c = self.db.cursor()
        c.execute(strSQL) 
        rtnStr = ''   
        for all in c.fetchall():
            rtnStr += (', '.join([str(i) for i in all]) + '\n')
        return rtnStr

    ###########################
    def deleteTable(self, tableName):
        """Delete a table from the database, given the name as a string.

        Args:
            | tableName (string): name of the table to be deleted.

        Returns:
            | None.

        Raises:
            | None.
        """
        c = self.db.cursor()
        self.executeSQL(c,'drop table if exists {}'.format(tableName))

    ###########################
    def getCursor(self):
        """Get the cursor from the database.

        Args:
            | None

        Returns:
            | (SQLite cursor): the cursor to be used in the operation.

        Raises:
            | None.
        """
        return self.db.cursor()

    ###########################
    def getCon(self):
        """Get a connection to the database.

        Args:
            | None

        Returns:
            | (SQLite database): database .

        Raises:
            | None.
        """
        return self.db

    ###########################
    def appendDFtoSqlOverwrite(self, dataFrame, tablename, keyName):
        """Append/overwrite a DataFrame to a named SQLite table.
        
        Before writing any rows to the database table, any entries with a matching
        column value (such as the primary key) are deleted. This is done because if 
        primary keys are enforced in the table, rows with any rows with matching 
        primary keys must be deleted.

        Args:
            | dataFrame (SQLite table): The dataFrame to be used in the operation.
            | tablename (string): Sqlite table name.
            | keyName (string): Dataframe/Sqlite primary key name.

        Returns:
            | None.

        Raises:
            | None.
        """
        # print('Removing existing enties before attempting to write')
        for item in dataFrame[keyName].tolist():
            strSQL = "delete from {} where {}='{}';".format(tablename, keyName, item)
            self.db.cursor().execute(strSQL)
        #append the data to the dataFrame in the database
        dataFrame.to_sql(tablename, self.db, if_exists='append', index=False)


In the example code following, three tables are created: `names`, `address` and `clubs`.  The `name` table has entries for linking to the `address` and `clubs` tables.  The database is trivial but does demonstrate the operation of the class defined above.

Create the database

In [3]:
db=dBase('test.db3')

Create the three tables.  Each table enforces a primary key on the table.  

In [4]:
db.execSQL("""create table if not exists names(
         nameID  INTEGER NOT NULL
        ,name TEXT DEFAULT('') 
        ,surname TEXT DEFAULT('') 
        ,count INTEGER  DEFAULT(0)
        ,addressID INTEGER  DEFAULT(0) 
        ,clubID INTEGER  DEFAULT(0) 
        ,constraint fk_addressID  FOREIGN KEY (addressID) 
          REFERENCES address(addressID)
        ,constraint fk_clubID  FOREIGN KEY (clubID) 
          REFERENCES clubs(clubID)
        ,constraint pk_nameID PRIMARY KEY (nameID)
      )""")

db.execSQL("""create table if not exists address(
         addressID  INTEGER NOT NULL
        ,street TEXT DEFAULT('') 
        ,number INTEGER  DEFAULT(0)
        ,constraint pk_addressID PRIMARY KEY (addressID)
       )""")

db.execSQL("""create table if not exists clubs(
         clubID  INTEGER NOT NULL
        ,clubname TEXT DEFAULT('') 
        ,constraint pk_clubID PRIMARY KEY (clubID)
       )""")

Set up Pandas dataframes with dummy data.  In this case the names are hard-coded to an address and a club entry.

In [5]:
#set up the addresses
address = pd.DataFrame([
    [0, 'Church Ave',5],
    [1, 'Schoeman Str',334],
    [2, 'Ajo Road', 8880]
    ],
    columns=['addressID','street','number'])

#set up the clubs
clubs = pd.DataFrame([
    [0, 'Chess Club'],
    [1, 'Flash Dancers'],
    ],
    columns=['clubID','clubname'])

#set up the names table, note how data is appended to the pre-created table
names = pd.DataFrame(columns=['nameID','name','surname','count','addressID','clubID'])
for cnt in range(0,3):
    tnames = pd.DataFrame([
        [cnt*3 + 0, 'John', 'Smith{}'.format(cnt),cnt,0,0],
        [cnt*3 + 1, 'Peter', 'Sellers{}'.format(cnt),cnt,1,1],
        [cnt*3 + 2, 'Max', 'Johnson{}'.format(cnt),cnt,2,0]
        ],
        columns=['nameID','name','surname','count','addressID','clubID'])
    names = names.append([tnames], ignore_index=True)


Write the dataframes to the database:

In [6]:
db.appendDFtoSqlOverwrite(address, 'address', 'addressID')
db.appendDFtoSqlOverwrite(clubs, 'clubs', 'clubID')
db.appendDFtoSqlOverwrite(names, 'names', 'nameID')

At this point you can inspect the SQLite database with a tool such as the `SQLite MAnager` running inside Foxpro.  This database can also now be processed in any other tool that has a SQLite interface.

Here follows some examples using SQLite directly on the database, as written above.

In [7]:
strSQL = 'select nameID, name, surname, count from names where count={};'.format(0)
print(strSQL)
print(db.queryToString(strSQL))

select nameID, name, surname, count from names where count=0;
0, John, Smith0, 0
1, Peter, Sellers0, 0
2, Max, Johnson0, 0



In [8]:
strSQL = """select names.name, names.surname, clubs.clubname from 
names inner join clubs on names.clubID=clubs.clubID;"""
print(strSQL)
print(db.queryToString(strSQL))

select names.name, names.surname, clubs.clubname from 
names inner join clubs on names.clubID=clubs.clubID;
John, Smith0, Chess Club
Peter, Sellers0, Flash Dancers
Max, Johnson0, Chess Club
John, Smith1, Chess Club
Peter, Sellers1, Flash Dancers
Max, Johnson1, Chess Club
John, Smith2, Chess Club
Peter, Sellers2, Flash Dancers
Max, Johnson2, Chess Club



In [9]:
strSQL = """select names.name, names.surname, clubs.clubname from 
names inner join clubs on names.clubID=clubs.clubID where 
clubs.clubname='Flash Dancers';"""
print(strSQL)
print(db.queryToString(strSQL))


select names.name, names.surname, clubs.clubname from 
names inner join clubs on names.clubID=clubs.clubID where 
clubs.clubname='Flash Dancers';
Peter, Sellers0, Flash Dancers
Peter, Sellers1, Flash Dancers
Peter, Sellers2, Flash Dancers



Now wipe all dataframes from memory and reload these again from the database

In [10]:
names = None
clubs = None
address = None

print(names)
print(clubs)
print(address)

None
None
None


In [11]:
names = pd.io.sql.read_sql('Select * from names',db.getCon(), index_col=None)
clubs = pd.io.sql.read_sql('Select * from clubs',db.getCon(), index_col=None)
address = pd.io.sql.read_sql('Select * from address',db.getCon(), index_col=None)

print(names)
print(clubs)
print(address)

   nameID   name   surname  count  addressID  clubID
0       0   John    Smith0      0          0       0
1       1  Peter  Sellers0      0          1       1
2       2    Max  Johnson0      0          2       0
3       3   John    Smith1      1          0       0
4       4  Peter  Sellers1      1          1       1
5       5    Max  Johnson1      1          2       0
6       6   John    Smith2      2          0       0
7       7  Peter  Sellers2      2          1       1
8       8    Max  Johnson2      2          2       0
   clubID       clubname
0       0     Chess Club
1       1  Flash Dancers
   addressID        street  number
0          0    Church Ave       5
1          1  Schoeman Str     334
2          2      Ajo Road    8880


## Python and [module versions, and dates](http://nbviewer.ipython.org/github/jrjohansson/scientific-python-lectures/blob/master/Lecture-0-Scientific-Computing-with-Python.ipynb)

In [12]:
%load_ext version_information
%version_information numpy, pandas

Software versions
Python 2.7.8 32bit [MSC v.1500 32 bit (Intel)]
IPython 3.0.0
OS Windows 7 6.1.7601 SP1
numpy 1.9.2
pandas 0.15.2
Sun Jun 28 09:58:37 2015 South Africa Standard Time